# Vit-Gpt2 

In [20]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 113.0 MB/s eta 0:00:00


In [ ]:
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
import torch
from PIL import Image

model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
#tokenizer = AutoTokenizer.from_pretrained(
# 'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
# bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]', pad_token='[PAD]', mask_token='[MASK]'
#)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


max_length = 20
num_beams = 9
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}

def predict_step(image_paths):
  images = []
  for image_path in image_paths:
    i_image = Image.open(image_path)
    if i_image.mode != "RGB":
      i_image = i_image.convert(mode="RGB")

    images.append(i_image)

  pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
  pixel_values = pixel_values.to(device)

  output_ids = model.generate(pixel_values, **gen_kwargs)

  preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
  preds = [pred.strip() for pred in preds]
  return preds


In [ ]:
predict_step(['/content/drive/MyDrive/IMG_3265.jpeg'])

['two men standing next to each other in a field']

# 한-영 번역 모델

## set up

In [ ]:
!pip install datasets

In [ ]:
!pip install sentencepiece

In [ ]:
!pip install evaluate

In [4]:
!pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 13.9 MB/s eta 0:00:00


In [5]:
!head -5 news_talk_en_ko_train_130000.tsv

head: cannot open 'news_talk_en_ko_train_130000.tsv' for reading: No such file or directory


## Dataset

In [6]:
from datasets import load_dataset

In [7]:
en_ko = load_dataset("bongsoo/news_talk_en_ko")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/bongsoo___csv/bongsoo--news_talk_en_ko-e7f00bc8f76f18d5/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
import pandas as pd

In [9]:
# 허깅페이스 데이터셋을 판다스 포맷으로 세팅
en_ko.set_format(type="pandas")

In [10]:
# 'train'키의 모든 행을 DataFrame df에 할당
df = en_ko["train"][:]

# 잘 담겼는지 확인한다.
df.head()

,Skinner's reward is mostly eye-watering.,스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.
0,Even some problems can be predicted.,심지어 어떤 문제가 발생할 건지도 어느 정도 예측이 가능하다.
1,Only God will exactly know why.,오직 하나님만이 그 이유를 제대로 알 수 있을 겁니다.
2,Businesses should not overlook China's dispute.,중국의 논쟁을 보며 간과해선 안 될 게 기업들의 고충이다.
3,Slow-beating songs often float over time.,박자가 느린 노래는 오랜 시간이 지나 뜨는 경우가 있다.
4,I can't even consider uninsured treatments.,보험 처리가 안 되는 비급여 시술은 엄두도 못 낸다.


In [11]:
example_0 = list(df.columns)
example_0

["Skinner's reward is mostly eye-watering.",
 '스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.']

In [12]:
example_0_df = pd.DataFrame({col: [value] for col, value in zip(('en', 'ko'), example_0)})
df.columns = ('en', 'ko')
en_ko_df = pd.concat([example_0_df, df],).reset_index(drop=True)
en_ko_df.head()

,en,ko
0,Skinner's reward is mostly eye-watering.,스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.
1,Even some problems can be predicted.,심지어 어떤 문제가 발생할 건지도 어느 정도 예측이 가능하다.
2,Only God will exactly know why.,오직 하나님만이 그 이유를 제대로 알 수 있을 겁니다.
3,Businesses should not overlook China's dispute.,중국의 논쟁을 보며 간과해선 안 될 게 기업들의 고충이다.
4,Slow-beating songs often float over time.,박자가 느린 노래는 오랜 시간이 지나 뜨는 경우가 있다.


In [13]:
from datasets import Dataset

In [14]:
dataset = Dataset.from_pandas(en_ko_df)

In [15]:
dataset

Dataset({
    features: ['en', 'ko'],
    num_rows: 1300000
})

In [16]:
# 각 데이터 셋의 샘플수를 정한다.
num_train = 1200000
num_valid = 90000
num_test = 10000

In [17]:
en_ko_df_train = en_ko_df.iloc[:num_train]
en_ko_df_valid = en_ko_df.iloc[num_train:num_train+num_valid]
en_ko_df_test = en_ko_df.iloc[-num_test:]

In [18]:
en_ko_df_train.to_csv("train.tsv", sep='\t', index=False)
en_ko_df_valid.to_csv("valid.tsv", sep='\t', index=False)
en_ko_df_test.to_csv("test.tsv", sep='\t', index=False)

data_files = {"train": "train.tsv", "valid": "valid.tsv", "test": "test.tsv"}
dataset =  load_dataset("csv", data_files=data_files, delimiter="\t")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-9b28fdb9eef240e4/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset

## Train

In [ ]:
!pip install transformers

In [21]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, load_metric
import numpy as np
import torch
import multiprocessing
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [22]:
model_ckpt = "KETI-AIR/ke-t5-base"
max_token_length = 64

In [23]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt) 

In [24]:
dataset['train'][10]['en'], dataset['train'][10]['ko']

('Any academic achievement requires constant repetition.',
 '어떤 학문이든지 일정의 성취를 이루기 위해서는 끊임없는 반복이 필요하다.')

In [25]:
tokenized_sample_en = tokenizer(dataset['train'][10]['en'], 
                                max_length=max_token_length, 
                                padding=True, truncation=True)
tokenized_sample_en

{'input_ids': [13941, 10114, 25542, 9361, 20526, 742, 32268, 12520, 3, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [28]:
def convert_examples_to_features(examples):
    model_inputs = tokenizer(examples['en'],
                             text_target=examples['ko'], 
                             max_length=max_token_length, truncation=True)
    
    return model_inputs

In [29]:
NUM_CPU = multiprocessing.cpu_count() 
NUM_CPU

2

In [30]:
tokenized_datasets = dataset.map(convert_examples_to_features, 
                                 batched=True, 
                                 # 이걸 쓰지 않으면 원 데이터 'en', 'ko'가 남아서
                                 # 아래서 콜레이터가 패딩을 못해서 에러남
                                 remove_columns=dataset["train"].column_names,
                                 num_proc=NUM_CPU) 

Map (num_proc=2):   0%|          | 0/1200000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/90000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [31]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1200000
    })
    valid: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 90000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
})

In [32]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

In [33]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [34]:
import evaluate
metric = evaluate.load("sacrebleu")

In [35]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    
    if isinstance(preds, tuple):
        preds = preds[0]
    
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    
    return result

In [39]:
training_args = Seq2SeqTrainingArguments(
    output_dir="chkpt",
    learning_rate=0.0005,
    weight_decay=0.01,
    per_device_train_batch_size=12,   # colab 학습 시 12로 수정 10시간 넘게 걸림.
    per_device_eval_batch_size=128,
    num_train_epochs=1,
    save_steps=500,
    save_total_limit=2,
    evaluation_strategy="epoch",
    logging_strategy="no",
    predict_with_generate=True,
    fp16=False,
    gradient_accumulation_steps=2,
    report_to="none" # Wandb 로그 끄기
)

In [40]:
trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [42]:
trainer.train() 

OutOfMemoryError: ignored

In [ ]:
trainer.save_model("./results")

In [ ]:
model_dir = "./results"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

model.cpu();

## Test

In [ ]:
input_text = [
    "Because deep learning frameworks are well developed, in these days, machine translation system can be built without anyone's help.",
    "This system was made by using HuggingFace's T5 model for a one day"
]

inputs = tokenizer(input_text, return_tensors="pt", 
                   padding=True, max_length=max_token_length)

In [ ]:
koreans = model.generate(
    **inputs,
    max_length=max_token_length,
    num_beams=5,
)

koreans.shape

In [ ]:
[ 
    tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(korean)) for korean in koreans
]

In [ ]:
from torch.utils.data import DataLoader

test_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=32, collate_fn=data_collator
)

In [ ]:
test_dataloader_iter = iter(test_dataloader)
test_batch = next(test_dataloader_iter)
test_input = { key: test_batch[key] for key in ('input_ids', 'attention_mask') }

In [ ]:
koreans = model.generate(
    **test_input,
    max_length=max_token_length,
    num_beams=5,
)

In [ ]:
labels =  np.where(test_batch.labels != -100, test_batch.labels, tokenizer.pad_token_id)
eng_sents = tokenizer.batch_decode(test_batch.input_ids, skip_special_tokens=True)[10:20]
references = tokenizer.batch_decode(labels, skip_special_tokens=True)[10:20]
preds = tokenizer.batch_decode( koreans, skip_special_tokens=True )[10:20]
for s in zip(eng_sents, references, preds):
    print('English   :', s[0])
    print('Reference :', s[1])
    print('Translated:', s[2])
    print('\n')